# Qtools tutorial 

## Enter Metadata and handle metadata

### Situation

We got a new device, we want to do a measurement on. These are the steps to enter the metadata

1. Add the device to the Metadatabase
2. Create metadata for the measurement
3. Setup the measurement
4. Run the measurement

**Remarks:**

- Steps 1. and 2. can also be done in one step

- During steps 3 and 4, more metadata is gathered with qtools.

### Add device to Metadatabase

Here, we create new metadata for the device, incorporate existing layouts, factories, etc. and save the metadata to the DB.

The helper function guides us through the creation of the metadata.

In [ ]:
from qtools_metadata.metadata import create_metadata_object
from qtools_metadata.device import Device

# Pass Device as argument, to create only the device
device = create_metadata_object(Device)

In [ ]:
from pprint import pprint
pprint(device)

In [ ]:
from qtools_metadata.metadata import save_metadata_object_to_db
save_metadata_object_to_db(device)
print(device.pid)

### Create metadata for the measurement

To create a complete Metadata set for a measurement, the same helper function is used.

In [ ]:
from qtools_metadata.metadata import Metadata

# create_metadata_object() is synonymous
metadata = create_metadata_object(Metadata)

In [ ]:
# save metadata to YAML file for future use
with open("metadata.yaml", "w") as f:
    metadata.to_yaml(f)

### Setup the measurement

### Run the measurement

Metadata is saved automatically to the DB.

### Load metadata from YAML

In [ ]:
from pprint import pprint
from qtools_metadata.metadata import Metadata

# Load metadata template from YAML file
with open("metadata.yaml", "r") as f:
    metadata: Metadata = Metadata.from_yaml(f)

# Change some stuff
metadata.measurement.name = "mynewmeasurement02"
metadata.measurement.comments = "a second measurement"

# Next measurement...